Heaviy adapted from Week5 tutorial

In [2]:

print("test import")
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import LongformerTokenizer, LongformerForSequenceClassification
import torch

test import


In [ ]:
device = torch.cuda.current_device() if torch.cuda.is_available() else "cpu" # This line checks if a GPU is available and sets the device to GPU (e.g., cuda:0) or CPU.
print(device)
#device = "cpu"

# Initialise the model and tokenizer to a pre-trained model. Suggestions: facebook/opt-350m, bigscience/bloom-560m

# model is made to classify sentence into 1/3 categories -> wa is a seq classification task
model = AutoModelForSequenceClassification.from_pretrained("allenai/longformer-base-4096", num_labels=3)
tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

#model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m").to(device)
#tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

0


Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Finetuning

In [4]:
import transformers
import pandas as pd
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder



fp = "../DATASETS/work_arrangements_development_set.csv"
df = pd.read_csv(fp)
df.drop("id", axis=1, inplace=True) #get rid of id column

le = LabelEncoder() # more efficient/relevant enum

df["label"] = le.fit_transform(df["y_true"])
df.rename(columns={"job_ad": "text"}, inplace=True)

label_map = dict(zip(le.classes_, le.transform(le.classes_)))
print(label_map)



{'Hybrid': np.int64(0), 'OnSite': np.int64(1), 'Remote': np.int64(2)}


In [5]:
#tokenised
data = Dataset.from_pandas(df)

data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)
data.set_format("torch")

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [8]:
training_args = transformers.TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,
    per_device_train_batch_size=1,
    logging_dir="./logs",
    report_to="none"
)

# === 6. Trainer and training ===
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=data,
    tokenizer=tokenizer
)

trainer.train()

/tmp/ipykernel_7570/839073864.py:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Trainer(


RuntimeError: CUDA error: unknown error
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [7]:
import torch
print(torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))
    print("Memory allocated:", torch.cuda.memory_allocated())

2.6.0+cu118
CUDA Available: True
Device: NVIDIA GeForce GTX 1650 Ti
Memory allocated: 3311861248


Eval